# AI-Powered Cat Care Assistant Demo

A demonstration of an AI assistant that combines vector similarity search with OpenAI embeddings and Claude for intelligent responses about cat care and veterinary appointments.

## Setup

First, we'll import our required libraries and initialize our clients.


In [35]:
from pinecone import Pinecone, ServerlessSpec
import os
import dotenv
from openai import OpenAI
from anthropic import Anthropic
import json

# Load environment variables
dotenv.load_dotenv()

# Initialize clients
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
anthropic_client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# Initialize Pinecone index
index = pc.Index("showcase-index")


## Define the tools

For each tool, we define a name, description, and input schema. The input schema is a JSON schema that describes the input for the tool

In [36]:
tools = [
    {
        "name": "search_cat_facts",
        "description": "Embeds a query and searches the vector database for relevant cat facts",
        "input_schema": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "The question or topic about cats to search for"
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "check_vet_availability",
        "description": "Checks available veterinarian appointment slots for the next 3 days",
        "input_schema": {
            "type": "object",
            "properties": {}
        }   
    },
    {
        "name": "book_appointment",
        "description": "Books a veterinarian appointment for a specific time",
        "input_schema": {
            "type": "object",
            "properties": {
                "day": {
                    "type": "string",
                    "description": "Day of the week"
                },
                "time": {
                    "type": "string",
                    "description": "Time slot"
                }
            },
            "required": ["day", "time"]
        }
    }

]

In [37]:
import anthropic
client = anthropic.Client()
MODEL_NAME = "claude-3-opus-20240229"

## 4. Tool Implementation
Implement the core functionality for each tool:

- ### search_cat_facts() 
Converts queries into embeddings and searches Pinecone for relevant cat facts.

- ### check_vet_availability()
Simulates checking available veterinary appointment slots.

- ### book_appointment()
Simulates booking a specific veterinary appointment.

- ### process_tool_call()
Routes tool calls to their appropriate functions.

In [38]:
def search_cat_facts(query):
    """
    Query the vector database for relevant cat facts
    """
    # Get embedding for the query
    query_embedding = openai_client.embeddings.create(
        model="text-embedding-3-small",
        input=[query]
    )
    
    # Search Pinecone
    results = index.query(
        vector=query_embedding.data[0].embedding,
        top_k=3,
        include_values=False,
        include_metadata=True
    )
    
    # Extract and return the relevant facts
    return [match['metadata']['text'] for match in results['matches']]

def check_vet_availability():
    print(f"Bipbip boop boop *** Fetching availability from the vet *** bipbip boop boop")
    availability = {
        "Monday": ["10:00-12:00", "12:00-14:00", "14:00-16:00"],
        "Tuesday": ["10:00-12:00", "12:00-14:00", "14:00-16:00"],
        "Wednesday": ["10:00-12:00", "12:00-14:00", "14:00-16:00"],
        "Thursday": ["10:00-12:00", "12:00-14:00", "14:00-16:00"],
        "Friday": ["10:00-12:00", "12:00-14:00", "14:00-16:00"]
    }
    
    return availability

def book_appointment(day, time):
    """Book a specific appointment slot"""
    print(f"Booking appointment for {day} at {time}")
    print(f"Bipbip boop boop *** Sending API request to the vet *** bipbip boop boop")
    print(f"Bipbip boop boop *** booking confirmed *** bipbip boop boop")
    return {
        "status": "confirmed",
        "appointment": {
            "day": day,
            "time": time
        },
        "booking_reference": "VET123"
    }

def process_tool_call(tool_name, tool_input):
    if tool_name == "search_cat_facts":
        return search_cat_facts(tool_input["query"])    
    elif tool_name == "check_vet_availability":
        return check_vet_availability()
    elif tool_name == "book_appointment":
        return book_appointment(tool_input["day"], tool_input["time"])

## 5. Conversation Management
Implements the conversation flow and tool usage logic:
- Maintains conversation history
- Handles tool calls from Claude
- Processes tool results
- Manages the back-and-forth between user and AI

In [41]:
import json

class ConversationManager:
    def __init__(self):
        self.conversation_history = []
    
    def add_message(self, role, content):
        self.conversation_history.append({"role": role, "content": content})
    
    def get_messages(self):
        return self.conversation_history
    
    def clear_history(self):
        self.conversation_history = []

def chatbot_interaction(user_message, conversation_manager=None):
    if conversation_manager is None:
        conversation_manager = ConversationManager()

    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")
    conversation_manager.add_message("user", user_message)

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        tool_choice={"type": "auto"},
        messages=conversation_manager.get_messages()
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        conversation_manager.add_message("assistant", response.content)
        conversation_manager.add_message("user", [
            {
                "type": "tool_result",
                "tool_use_id": tool_use.id,
                "content": str(tool_result),
            }
        ])

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=conversation_manager.get_messages()
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")
        

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response,conversation_manager

## 6. Demo Usage
Example interactions demonstrating the assistant's capabilities:
1. Querying about cat anatomy
2. Checking vet availability
3. Booking an appointment

In [42]:
if __name__ == "__main__":
    # Create a conversation manager to maintain state
    conversation = ConversationManager()
    
    # First question about cat toes
    response1, conversation = chatbot_interaction(
        "How many toes does a cat have?",
        conversation
    )
    
    # Second question about abnormal toes and checking availability
    response2, conversation = chatbot_interaction(
        "Oh, my cat only has 2 toes on each paw, do you see any times for the vet that I can book?",
        conversation
    )
    
    # Third question to book specific appointment
    response3, conversation = chatbot_interaction(
        "Perfect! Can you book 12:00 for me on Thursday?",
        conversation
    )


User Message: How many toes does a cat have?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='<thinking>\nThe user is asking a factual question about cats, specifically how many toes they have. The most relevant tool for this query is search_cat_facts, which allows searching a database of cat facts for information related to a specific question or topic.\n\nThe search_cat_facts tool requires a "query" parameter, which in this case can be directly inferred from the user\'s question. No other tools are needed, as this query can likely be answered with a single search.\n</thinking>', type='text'), ToolUseBlock(id='toolu_01L6M882cTYKpQg8uiRRgovG', input={'query': 'How many toes does a cat have?'}, name='search_cat_facts', type='tool_use')]

Tool Used: search_cat_facts
Tool Input:
{
  "query": "How many toes does a cat have?"
}

Tool Result:
[
  "Cats have five toes on each front paw, but only four toes on each back paw.",
  "Cats are sometimes born with extra toes. This 